In [3]:
#necessary imports 
import csv
import pandas as pd
import os
import re
import string
import collections


import numpy as np
from itertools import groupby
from sklearn.feature_extraction import text
import nltk
from nltk.corpus import stopwords
import datetime
import ast

#do in terminal python3 -m pip install --upgrade pandas
import statistics

import string
folder_prefix = '/home/jovyan/work/'

In [4]:
original_df = pd.read_pickle(folder_prefix + 'nowdata/charters_2015.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/nowdata/charters_2015.pkl'

In [80]:
new_ser = []
for li in original_df['WEBTEXT']:
    if li == "":
        new_ser.append([])
    else:
        new_ser.append(li)

In [81]:
original_df['WEBTEXT'] = new_ser

In [83]:
original_df['WEBTEXT'][0]

[('http://www.maef.net/',
  'False',
  '0',
  "Evening Academy\nAlabama's first tuition-free public charter school serves high school students in grades 9-12 from Mobile, Baldwin, and Washington Counties. ACCEL provides a challenging college-preparatory curriculum, individualized instruction, small class sizes, and engaging use of technology in a safe, supportive environment to ensure students graduate college and career ready. Enrollment is now OPEN\nLearn More\nWelcome to the Mobile Area Education Foundation\n75K Degrees\nEvidence2Success\nGraduate Ready\nYes We Can: Building a Blueprint for Equity and Excellence in Mobile’s Schools  \nACCELerate Day and Evening Academy\nEducation Commission\nEngaging Youth Through Engineering\nHarold Dodge Fund-A-S.T.A.R.\nPartners in Education\nReading Buddy\nResearch Alliance for Multiple Pathways (RAMP)\nSuperintendent’s Student Advisory Council\nVital Link 2.0\nThe Mobile Area Education Foundation is a nonprofit organization dedicated to improvi

In [ ]:
original_df['WEBTEXT'][0]

In [38]:
keywords = ['values', 'academics', 'academic', 'skills', 'skill', 'purpose', 'purposes',
                       'direction', 'mission', 'vision', 'visions', 'missions',
                       'ideals', 'cause', 'causes', 'curriculum', 'curricular',
                       'method', 'methods', 'pedagogy', 'pedagogical', 'pedagogies', 'approach', 'approaches', 'model', 'models', 'system', 'systems',
                       'structure', 'structures', 'philosophy', 'philosophical', 'philosophies', 'beliefs', 'believe', 'belief',
                       'principles', 'principle', 'creed', 'creeds', 'credo', 'moral', 'morals', 'morality', 'history', 'histories', 'our story',
                       'the story', 'school story', 'background', 'backgrounds', 'founding', 'founded', 'foundation', 'foundations', 'foundational',
                       'established','establishment', 'our school began', 'we began',
                       'doors opened', 'school opened', 'about us', 'our school', 'who we are',
                       'identity', 'identities', 'profile', 'highlights']

In [39]:
keywords_values = {'values':2, 'academics':1, 'academic':1, 'skills':1, 'skill':1, 'purpose':2, 'purposes':2,
                       'direction':1, 'mission':2, 'vision':2, 'visions':2, 'missions':2,
                       'ideals':2, 'cause':1, 'causes':1, 'curriculum':2, 'curricular':2,
                       'method':1, 'methods':1, 'pedagogy':2, 'pedagogical':1, 'pedagogies':1, 'approach':1, 'approaches':1, 'model':2, 'models':2, 'system':2, 'systems':2,
                       'structure':1, 'structures':1, 'philosophy':2, 'philosophical':2, 'philosophies':2, 'beliefs':2, 'believe':2, 'belief':2,
                       'principles':2, 'principle':2, 'creed':2, 'creeds':2, 'credo':2, 'moral':2, 'morals':2, 'morality':2, 'history':1, 'histories':1, 'our story':1,
                       'the story':1, 'school story':1, 'background':1, 'backgrounds':1, 'founding':1, 'founded':1, 'foundation':1, 'foundations':1, 'foundational':1,
                       'established':1,'establishment':1, 'our school began':1, 'we began':1,
                       'doors opened':1, 'school opened':1, 'about us':2, 'our school':1, 'who we are':1,
                       'identity':1, 'identities':1, 'profile':1, 'highlights':2}

In [40]:
# Hybrid approach

# Separate keywords to be treated differently
small_keywords = []
large_keywords = []

for entry in keywords:
    small_keywords.append(entry) if len(entry.split()) < 3 else large_keywords.append(entry)

large_words = [entry.split() for entry in large_keywords] # list words for each large dict entry
large_lengths = [len(x) for x in large_words]
large_first_words = [x[0] for x in large_words] # first words of each large entry in dict

In [41]:
def filter_pages2(school_pages, MIN_HITCOUNT = 1, MAX_NUMPAGES = 250, AGGRO = False, is_set = False):
    """Filters page text with hit count at least min hit count if school has more than MAX_NUMPAGES distinct pages else unfiltered of pages is returned.
    
    Returns max_numpages pages with priority given to higher hitscore and then lower page depth(even when AGGRO is TRUE). Boolean value returned is to help generate WEBTEXT_METHOD later.
    school_pages: entry of 'webtext' column
    is_set: True if school_pages is set of pages
    aggro: When true, only pages that have >= MIN_HITCOUNT hits pass. Only resort to CMO pages when no pages pass
    """
    if not is_set:
        school_pages = set([p for p in school_pages])
#     if len(pages) <= MAX_NUMPAGES:
#         return ([(p.url, p.boo, p.depth, p.text) for p in pages], 0)
    all_tuples = []
    filtered_num = 0 # number of pages that passed the hitscore requirement
    filtered = []
    max_hc = -1
    min_depth = 99999
    for p in school_pages:
        hit_count = dict_count2(p)
        if hit_count >= MIN_HITCOUNT:
            filtered.append(hit_count, p)
            filtered_num += 1
#         if max_hc < hit_count:
#             max_hc = hit_count
#         if min_depth > int(p.depth):
#             min_depth = int(p.depth)
        # maintain list containing all pages and corresponding hit scores
        all_tuples.append((hit_count, (p.url, p.boo, p.depth, p.text)))
    if not aggro and filtered_num and filtered_num <= MAX_NUMPAGES:
            return ([t[1] for t in filtered], False)        
    all_tuples = [(t[0] - .00001*int(t[1][2]), t[1]) for t in all_tuples] # prepare list to be heapified
    if aggro:
        all_tuples = filtered
    # priority number is hit_count - .00001*page.depth so pages with high hitscores are prioritized followed by low page depths
    filtered = [t[1] for t in q.nlargest(MAX_NUMPAGES, all_tuples)]
    return (filtered, filtered_num == False)

In [42]:
def dict_count2(text):

    """Hybrid of dict_count and dict_count1. 
    
    Uses dict_count1 approach to count matches for entries with > 2 words in keywords.
    Uses dict_count approach for all other entries.
    """

    counts = 0 # hitscore
    splitted_phrase = re.split('\W+|_', text.lower()) # Remove punctuation with regex that keeps only letters and spaces

    for length in range(1, 3):
        if len(splitted_phrase) < length:
            continue # If text chunk is shorter than length of dict entries being matched, there are no matches.
        for i in range(len(splitted_phrase) - length + 1):
            entry = ' '.join(splitted_phrase[i:i+length]) # Builds chunk of 'length' words without ending space
            if entry in keywords:
                counts += keywords_values[entry]
    mask = [[word == entry for word in splitted_phrase] for entry in large_first_words]
    indices = np.transpose(np.nonzero(mask))
    for ind in indices:
        if ind[1] <= (len(splitted_phrase) - large_lengths[ind[0]]) and large_words[ind[0]] == splitted_phrase[ind[1] : ind[1] + large_lengths[ind[0]]]:
            counts += keywords_values[large_keywords[ind[0]]]
    return counts

In [43]:
def getKey(item):
    return item[0]

In [44]:
def takeSecond(elem):
    return elem[1]

In [45]:
def takeThird(elem):
    return elem[2]

In [88]:
num = 0

def filter_pages(li_tuples, MIN_HITCOUNT, MAX_NUMPAGES):
    """
    Takes in a list of quadruples
    string texts from a school. Most likely from the WEBTEXT column
    For the row, the function returns the top 250 pages of the list who have the highest hitcount    
    """
    global num
    
    #just keep track of the hit count
    
    #turn the list of quadruples into a list of just string texts  
    
    if len(li_tuples) == 0: #if taking in an emoty list, we return an empty list
        return li_tuples
    
    school_pages = []
    
    for tup in li_tuples:
        if len(tup) == 4:
            school_pages.append([tup[2], tup[3]])
    
    #depth = 0 #index, gow far in they are, depth in school_pages[1st part of tuple]
    li_pairs = []
    #index = 0
    for depth, page in school_pages:
        hit_count = dict_count2(page.lower())
        if hit_count >= MIN_HITCOUNT:
            li_pairs.append([page, hit_count - int(depth)])
        
        #index+=1
    
    #sort the tuples/sublists by highest to lowest hit count
    #take the top 250 , or less is len(school_pages) < 250
    
    #To do ascending on one and descending on the other, 
    #two calls: list5.sort(key=lambda vertex: vertext, reverse=True) list5.sort(key=lambda vertex: degree(vertext))
    
    sorted_tuples = sorted(li_pairs,  reverse = True) #ascending depth #prioritize pages closest to  0th depth
       
    filtered_tuples = []
    if len(sorted_tuples) < MAX_NUMPAGES:
        filtered_tuples = sorted_tuples
    else:
        filtered_tuples = sorted_tuples[:MAX_NUMPAGES]
    
    #get the page at the correpsonding index (tup[1]) from school_pages
    final_pages = [tup[0] for tup in filtered_tuples] 
    if final_pages is not None :
        m = 0
        #logging.info("row done : " + str(num))
    else:
        #logging.info("row is None : " + str(num))
        final_pages = []
    num = num + 1
    
    return final_pages

In [77]:
li_pairs= [["1500-0", 1500, 0], ["1502-1", 1502, 1], ["5000-3", 5000, 3],  ["1510-9", 1510, 9],  ["1510-30", 1510, -30] ]

In [78]:
dub = [[tup[0], takeSecond(tup) - takeThird(tup)] for tup in li_pairs]
print(dub)
sorted_tuples = sorted(dub, key = takeSecond, reverse = True) 
print(sorted_tuples)


[['1500-0', 1500], ['1502-1', 1501], ['5000-3', 4997], ['1510-9', 1501], ['1510-30', 1540]]
[['5000-3', 4997], ['1510-30', 1540], ['1502-1', 1501], ['1510-9', 1501], ['1500-0', 1500]]


In [73]:
# li_pairs.sort(key=lambda pair: takeSecond(pair), reverse = True) #descending hitcount 
# print(li_pairs)
#sort first by count (max first), second by depth (how far deep web page is in the site) (min first)
sorted_tuples = sorted(li_pairs, key = lambda pair: takeSecond(pair) - takeThird(pair), reverse = True) 
print(li_pairs)

[['hi', 1500, 0], ['hi', 1502, 1], ['hi', 5000, 3], ['hi', 1510, 9], ['hi', 1510, -30]]


In [60]:
print(sort_hit)
print(sorted_tuples)


[['hi', 5000, 3], ['hi', 1500, 0], ['hi', 1500, 1], ['hi', 1500, 9]]
[['hi', 1500, 0], ['hi', 1500, 1], ['hi', 5000, 3], ['hi', 1500, 9]]


In [89]:
small_df = original_df[:500]

In [90]:
small_df['WEBTEXT'] = small_df['WEBTEXT'].apply(lambda row_li: filter_pages(row_li, 1, 250))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
small_df['WEBTEXT']

0      [ut Reading Buddy\nThe Reading Buddy program i...
1                                                     []
2      [Select a School...\nSelect a School\nKetchika...
3      [l Enrollment Homeschool\nTitle IX: Assuring G...
4      [Select a School...\nSelect a School\nAbbott L...
5                                                     []
6                                                     []
7                                                     []
8      [eer Info\nWCC\nWPG\nBrowse: \nHome\n» \nWPG\t...
9                                                     []
10     [es\nLOGIN\nFrontier's Purpose: "Helping paren...
11                                                    []
12                                                    []
13     [Select a School...\nSelect a School\nAbbott L...
14     [Select a School...\nSelect a School\nAbbott L...
15     [Juneau Community Charter School \nJCCS \nA Le...
16                                                    []
17     [Fireweed Academy\n995 S

In [52]:
filt_250 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_250.pkl')

In [55]:
filt_250['WEBTEXT'][0]

['August 3, 2017\nACCEL Informational Meetings\nEvent\nMarch 25, 2017\nACCEL is now ACCEPTING Applications!\nEvent\nFebruary 23, 2017\nMobile Area News / Media\nArticle\nCategory\nDate\nWhat\'s Working\nACCEL Day and Evening Academy\nMay 31, 2018\nThe Accelerator (8/25)\nACCEL Day and Evening Academy\nSeptember 11, 2017\nU.S. education secretary visits Mobile\nACCEL Day and Evening Academy\nSeptember 1, 2017\nNonprofit prepares to open state’s first charter school\nACCEL Day and Evening Academy\nAugust 17, 2017\nNew Student Orientation\nACCEL Day and Evening Academy\nAugust 3, 2017\nMobile gets Alabama\'s first charter high school\nACCEL Day and Evening Academy\nAugust 3, 2017\nAM/NS Calvert STEM Summer Camp\nJuly 25, 2017\nPress Release: ACCEL Joins Cohort to Reimagine K-12 Learning\nACCEL Day and Evening Academy\nMay 3, 2017\nAlabama\'s first charter school launches in Mobile County\nACCEL Day and Evening Academy\nApril 26, 2017\nAlabama\'s first charter school will be in Mobile\nACC

In [56]:
small_df['WEBTEXT'][0]

["Evening Academy\nAlabama's first tuition-free public charter school serves high school students in grades 9-12 from Mobile, Baldwin, and Washington Counties. ACCEL provides a challenging college-preparatory curriculum, individualized instruction, small class sizes, and engaging use of technology in a safe, supportive environment to ensure students graduate college and career ready. Enrollment is now OPEN\nLearn More\nWelcome to the Mobile Area Education Foundation\n75K Degrees\nEvidence2Success\nGraduate Ready\nYes We Can: Building a Blueprint for Equity and Excellence in Mobile’s Schools  \nACCELerate Day and Evening Academy\nEducation Commission\nEngaging Youth Through Engineering\nHarold Dodge Fund-A-S.T.A.R.\nPartners in Education\nReading Buddy\nResearch Alliance for Multiple Pathways (RAMP)\nSuperintendent’s Student Advisory Council\nVital Link 2.0\nThe Mobile Area Education Foundation is a nonprofit organization dedicated to improving our local public schools. Founded in 1992,

In [ ]:
fitered_df = original_df[['NCESSCH', 'WEBTEXT']]

In [ ]:
filt_df = pd.read_csv(folder_prefix + 'nowdata/parsing/filtered_df.csv', sep="\t", low_memory=False, encoding="utf-8")

In [ ]:
len(filt_df['WEBTEXT'][0])

In [ ]:
filt_df['WEBTEXT'] = filt_df['WEBTEXT'].fillna('0')
filt_df['WEBTEXT'] = filt_df['WEBTEXT'].apply(ast.literal_eval) 

In [ ]:
len(filt_df['WEBTEXT'][0])

In [ ]:
filt_df['WEBTEXT'][0][8]

In [ ]:
#see if Nan's on filt_df = original_df Nan's


In [ ]:
filt_df['WEBTEXT'][9] == []

In [ ]:
original_df['NCESSCH'][0] == filt_df['NCESSCH'][0]

In [ ]:
filt_df['WEBTEXT'][1] == 0

In [ ]:
original_df['WEBTEXT'][1] == ''

In [ ]:
len(original_df['WEBTEXT'][0])

In [ ]:
num_pgs_list = []
for li in original_df['WEBTEXT']:
    num_pgs_list.append(len(li))

In [ ]:
max(num_pgs_list)

In [ ]:
i = 0
for num in num_pgs_list:
    if num > 250:
        i+=1
print(i) # only 238 are greater than 250
    

In [ ]:
None == None


In [ ]:
import math
x=float('nan')
math.isnan(x)

In [ ]:
filt_df['WEBTEXT'][1] == x


In [ ]:
(filt_df['NCESSCH'] == original_df['NCESSCH']).all()

In [ ]:
#just check if the places where I have put 0 (was initially NaN) in WEBTEXT column in filt_df 
i = 0
num_incorrect = 0
for li in filt_df:
    if li == 0 and not (original_df['WEBTEXT'][i] == ""):
        print(num_incorrect)
        num_incorrect+=1
    
    

In [ ]:
num_incorrect

In [ ]:
nan_count = 0
for li in filt_df['WEBTEXT']:
    if li == 0:
        nan_count+=1

print(nan_count)
        

In [ ]:
li_count = 0
for li in filt_df['WEBTEXT']:
    if li == []:
        li_count+=1

print(li_count)
        

In [ ]:
nan_count + li_count

In [ ]:
(type(filt_df['WEBTEXT'][9]) == list) and filt_df['WEBTEXT'][9]!= []

In [ ]:
actual_list_count = 0
for li in filt_df['WEBTEXT']:
    if (type(li) == list) and li!= []:
        actual_list_count+=1

print(actual_list_count)

In [ ]:
b = 0
for li in original_df['WEBTEXT']:
    if li == "":
        b+=1

print(b)
        

In [ ]:
#so Nan's are in the right place, stuff is filtered

In [28]:
filt_df = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_10.pkl')

In [44]:
orig_df = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_250.pkl')

In [6]:
len(filt_df['WEBTEXT'][0])

10

In [7]:
num_pgs_list = []
for li in filt_df['WEBTEXT']:
    num_pgs_list.append(len(li))

In [8]:
i = 0
for num in num_pgs_list:
    if num > 250:
        i+=1
print(i) # only 238 are greater than 250
    

0


In [12]:
max(num_pgs_list)

10

In [18]:
len_list = []
ind = 0
for li in filt_df['WEBTEXT']:
    if (len(li) == 0):
        len_list.append(ind)
    ind+=1

In [19]:
empty = []
ind_1 = 0
for li in filt_df['WEBTEXT']:
    if (li == ""):
        empty.append(ind_1)
    ind_1+=1

In [20]:
len_list == empty

False

In [21]:
set(len_list) == set(empty)

False

In [22]:
for i in len_list:
    if i not in empty:
        print(i)

9
11
35
203
316
371
372
414
471
485
495
518
538
583
623
626
627
645
696
697
709
712
713
716
719
749
763
806
807
810
821
822
823
824
831
834
846
851
854
856
866
873
881
1087
1088
1123
1128
1143
1154
1158
1192
1205
1304
1321
1339
1354
1365
1372
1440
1450
1452
1455
1514
1554
1557
1562
1590
1626
1627
1645
1647
1648
1649
1650
1683
1684
1685
1691
1697
1708
1727
1732
1742
1751
1754
1755
1769
1773
1782
1785
1807
1825
1829
1833
1841
1865
1867
1873
1875
1878
1883
1889
1893
1894
1896
1908
1911
1913
1934
1943
1945
1960
1968
1990
1991
1999
2009
2021
2027
2034
2045
2049
2069
2107
2128
2164
2190
2208
2247
2306
2326
2330
2332
2341
2352
2356
2363
2368
2369
2377
2431
2436
2449
2532
2577
2589
2598
2614
2615
2618
2621
2633
2637
2686
2702
2726
2755
2805
2807
2836
2864
2890
2899
2932
2969
3001
3003
3011
3013
3106
3139
3144
3147
3148
3173
3190
3202
3204
3232
3304
3306
3307
3308
3310
3311
3312
3313
3314
3315
3318
3319
3330
3331
3397
3412
3481
3482
3483
3488
3495
3497
3508
3511
3517
3518
3519
3520
3521
3530
35

In [24]:
filt_df['WEBTEXT'][11]

[]

In [29]:
new_ser = []
for li in filt_df['WEBTEXT']:
    if li == "":
        new_ser.append([])
    else:
        new_ser.append(li)

In [30]:
len(new_ser)


10965

In [31]:
filt_df['WEBTEXT'] = new_ser

In [32]:
filt_df

,NCESSCH,WEBTEXT
0,1.001970e+10,"[August 3, 2017\nACCEL Informational Meetings\..."
1,2.000010e+10,[]
2,2.001500e+10,[Select a School...\nSelect a School\nKetchika...
3,2.001500e+10,[l Enrollment Homeschool\nTitle IX: Assuring G...
4,2.001800e+10,[Select a School...\nSelect a School\nAbbott L...
5,2.001800e+10,[]
6,2.001800e+10,[]
7,2.001800e+10,[]
8,2.001800e+10,[Browse: \nHome\n» \nProgram\n» \nCurriculum\t...
9,2.001800e+10,[]


In [36]:
index = 0
for li in filt_df['WEBTEXT']:
    if li != orig_df['WEBTEXT'][index]:
        if (len(orig_df['WEBTEXT'][index]) > 0) and li == []:
            print(index)
    index+=1

In [37]:
filt_df.to_pickle(folder_prefix + "nowdata/parsing/filtered_10_1.pkl")

In [38]:
filt_250 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_250.pkl')

In [39]:
filt_10 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_10_1.pkl')

In [40]:
filt_10

,NCESSCH,WEBTEXT
0,1.001970e+10,"[August 3, 2017\nACCEL Informational Meetings\..."
1,2.000010e+10,[]
2,2.001500e+10,[Select a School...\nSelect a School\nKetchika...
3,2.001500e+10,[l Enrollment Homeschool\nTitle IX: Assuring G...
4,2.001800e+10,[Select a School...\nSelect a School\nAbbott L...
5,2.001800e+10,[]
6,2.001800e+10,[]
7,2.001800e+10,[]
8,2.001800e+10,[Browse: \nHome\n» \nProgram\n» \nCurriculum\t...
9,2.001800e+10,[]


In [41]:
new_ser = []
for li in filt_250['WEBTEXT']:
    if li == "":
        new_ser.append([])
    else:
        new_ser.append(li)

In [42]:
filt_250['WEBTEXT'] = new_ser

In [43]:
filt_250

,NCESSCH,WEBTEXT
0,1.001970e+10,"[August 3, 2017\nACCEL Informational Meetings\..."
1,2.000010e+10,[]
2,2.001500e+10,[Select a School...\nSelect a School\nKetchika...
3,2.001500e+10,[l Enrollment Homeschool\nTitle IX: Assuring G...
4,2.001800e+10,[Select a School...\nSelect a School\nAbbott L...
5,2.001800e+10,[]
6,2.001800e+10,[]
7,2.001800e+10,[]
8,2.001800e+10,[Browse: \nHome\n» \nProgram\n» \nCurriculum\t...
9,2.001800e+10,[]


In [46]:
index = 0
for li in filt_250['WEBTEXT']:
    if li != orig_df['WEBTEXT'][index]:
        if (len(orig_df['WEBTEXT'][index]) > 0) and li == []:
            print(index)
    index+=1

In [47]:
index

10965

In [48]:
filt_250.to_pickle(folder_prefix + "nowdata/parsing/filtered_250_1.pkl")

In [50]:
filt_250 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_250_1.pkl')

In [52]:
filt_250.to_pickle(folder_prefix + "nowdata/parsing/filtered_250.pkl")

In [53]:
filt_10 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_10_1.pkl')

In [54]:
filt_10.to_pickle(folder_prefix + "nowdata/parsing/filtered_10.pkl")

In [103]:
filt_10 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_10.pkl')

In [104]:
filt_10

,NCESSCH,WEBTEXT
0,1.001970e+10,[ut Reading Buddy\nThe Reading Buddy program i...
1,2.000010e+10,[]
2,2.001500e+10,[Select a School...\nSelect a School\nKetchika...
3,2.001500e+10,[l Enrollment Homeschool\nTitle IX: Assuring G...
4,2.001800e+10,[Select a School...\nSelect a School\nAbbott L...
5,2.001800e+10,[]
6,2.001800e+10,[]
7,2.001800e+10,[]
8,2.001800e+10,[eer Info\nWCC\nWPG\nBrowse: \nHome\n» \nWPG\t...
9,2.001800e+10,[]


In [97]:
filt_250 = pd.read_pickle(folder_prefix + 'nowdata/parsing/filtered_250.pkl')

In [98]:
filt_250

,NCESSCH,WEBTEXT
0,1.001970e+10,[ut Reading Buddy\nThe Reading Buddy program i...
1,2.000010e+10,[]
2,2.001500e+10,[Select a School...\nSelect a School\nKetchika...
3,2.001500e+10,[l Enrollment Homeschool\nTitle IX: Assuring G...
4,2.001800e+10,[Select a School...\nSelect a School\nAbbott L...
5,2.001800e+10,[]
6,2.001800e+10,[]
7,2.001800e+10,[]
8,2.001800e+10,[eer Info\nWCC\nWPG\nBrowse: \nHome\n» \nWPG\t...
9,2.001800e+10,[]


In [4]:
print("hi")

hi


In [99]:
lengths_250 = []
for li in filt_250['WEBTEXT']:
    lengths_250.append(len(li)) 

In [102]:
print("Max 250 : ", max(lengths_250))
print("Min 250 : ", min(lengths_250))
print("Median 250 : ", statistics.median(lengths_250))
print("Mean 250 : ", statistics.mean(lengths_250))
print("Mode 250 : ", statistics.mode(lengths_250))
print("Std dev 250 : ", statistics.stdev(lengths_250))

Max 250 :  250
Min 250 :  0
Median 250 :  1
Mean 250 :  18.987414500683993
Mode 250 :  0
Std dev 250 :  39.39090507128535


In [105]:
lengths_10 = []
for li in filt_10['WEBTEXT']:
    lengths_10.append(len(li)) 

In [106]:
print("Max 10 : ", max(lengths_10))
print("Min 10 : ", min(lengths_10))
print("Median 10 : ", statistics.median(lengths_10))
print("Mean 10 : ", statistics.mean(lengths_10))
print("Mode 10 : ", statistics.mode(lengths_10))
print("Std dev 10 : ", statistics.stdev(lengths_10))

Max 10 :  10
Min 10 :  0
Median 10 :  1
Mean 10 :  4.1820337437300505
Mode 10 :  0
Std dev 10 :  4.642811312667594
